In [1]:
import boto3

In [2]:
with open('hello.py','r') as f:
    function_code = f.read()


In [3]:
print(function_code)

def lambda_handler(event,context):
    return "hello world"


In [4]:
function_name = 'HelloWorld2'
runtime='python3.8'
handler='lambda_function.lambda_handler' #second part is the function name


In [5]:
import json
iam_client=boto3.client('iam',region_name='us-east-1')

In [6]:
#iam policy includes {service - permission} pairs


lambda_execution_policy = {
    'Version':'2012-10-17',
    "Statement": [
        {
            "Effect": "Allow",
            "Action": ["logs:CreateLogGroup",
                       "logs:CreateLogStream",
                       "logs:PutLogStream"],
            "Resource": "arn:aws:logs:*:*:*"
        },
    ]
}

In [7]:
lambda_execution_policy

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Action': ['logs:CreateLogGroup',
    'logs:CreateLogStream',
    'logs:PutLogStream'],
   'Resource': 'arn:aws:logs:*:*:*'}]}

In [9]:
role_name = 'LambdaExecutionRole'

In [11]:
#a role is assumed by a resource. Lambda resource in this example.
role_response = iam_client.create_role(RoleName=role_name,
                                       AssumeRolePolicyDocument=json.dumps(
                                       {
                                            'Version':'2012-10-17',
                                           'Statement':[
                                               {
                                               'Effect':'Allow',
                                                   "Principal":{'Service':"lambda.amazonaws.com"},
                                                   'Action':'sts:AssumeRole'
                                               }
                                           ]
                                       } 
                                       ))


EntityAlreadyExistsException: An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name LambdaExecutionRole already exists.

In [12]:
role_response

{'Role': {'Path': '/',
  'RoleName': 'LambdaExecutionRole',
  'RoleId': 'AROA2U4Z22B57VFHHV7TO',
  'Arn': 'arn:aws:iam::732077674619:role/LambdaExecutionRole',
  'CreateDate': datetime.datetime(2023, 11, 8, 4, 33, 9, tzinfo=tzutc()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Principal': {'Service': 'lambda.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]}},
 'ResponseMetadata': {'RequestId': '9ac65d7e-fe54-4f52-a1b9-9b77cd438428',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9ac65d7e-fe54-4f52-a1b9-9b77cd438428',
   'content-type': 'text/xml',
   'content-length': '792',
   'date': 'Wed, 08 Nov 2023 04:33:08 GMT'},
  'RetryAttempts': 0}}

In [14]:
policy_name='LambdaExecutionPolicy'

In [15]:
#linking a role to a policy

iam_client.put_role_policy(RoleName=role_name,PolicyName=policy_name,
                           PolicyDocument = json.dumps(lambda_execution_policy))

{'ResponseMetadata': {'RequestId': 'c0b08c74-90c5-4359-9333-f229132f592c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c0b08c74-90c5-4359-9333-f229132f592c',
   'content-type': 'text/xml',
   'content-length': '206',
   'date': 'Wed, 08 Nov 2023 04:37:43 GMT'},
  'RetryAttempts': 0}}

In [17]:
role_arn = role_response['Role']['Arn']
role_arn

'arn:aws:iam::732077674619:role/LambdaExecutionRole'

In [ ]:
##### the policy and role creation can be done on the console.
#instead of doing on boto3

In [20]:
import zipfile
import io

In [28]:
lambda_client=boto3.client('lambda',region_name='us-east-1')

In [36]:
with io.BytesIO() as deployment_package:
    with zipfile.ZipFile(deployment_package,'w') as zipf:
        zipf.writestr('lambda_function.py',function_code)
    
    create_function_response = lambda_client.create_function(
    FunctionName=function_name,
    Runtime=runtime,
    Role = role_arn,
    Handler = handler,
    Code = {'ZipFile':deployment_package.getvalue()}
    )

        


In [58]:
invoke_response = lambda_client.invoke(FunctionName=function_name)

In [56]:
invoke_response

{'ResponseMetadata': {'RequestId': '7bb0e389-84b3-4c75-8427-b016ea6dec5d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 08 Nov 2023 05:26:42 GMT',
   'content-type': 'application/json',
   'content-length': '13',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7bb0e389-84b3-4c75-8427-b016ea6dec5d',
   'x-amzn-remapped-content-length': '0',
   'x-amz-executed-version': '$LATEST',
   'x-amzn-trace-id': 'root=1-654b1c12-6f28e85d19715e35307f18b5;sampled=0;lineage=da5a4530:0'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'ExecutedVersion': '$LATEST',
 'Payload': <botocore.response.StreamingBody at 0x11051c610>}

In [62]:
payload = invoke_response['Payload'].read()
payload.decode('utf-8')

''

In [63]:
lambda_client.delete_function(FunctionName=function_name)

{'ResponseMetadata': {'RequestId': '7fe60277-a23d-4b08-b7bc-ce3520069daf',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Wed, 08 Nov 2023 05:28:26 GMT',
   'content-type': 'application/json',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7fe60277-a23d-4b08-b7bc-ce3520069daf'},
  'RetryAttempts': 0}}